In [74]:
import json
import os
import ast
import pandas as pd
os.chdir('/Users/sanjanarosario/Downloads/yelp_dataset')
rest_reviews=pd.read_csv('restaurant_reviews_newcols.csv',nrows=100000)
business_details=pd.read_csv('businesses_filtered.csv')

In [87]:
#selecting only desired columns
business_details=business_details[['business_id','name','stars','review_count']]
documents=rest_reviews[['business_id','text']]

In [88]:
#Must aggregate reviews for each restaurant
import collections as c
only_rests=set(rest_reviews['business_id'])
final_output=c.defaultdict()

def lolwut(df):
    for i in only_rests:
        emptylist=''
        temp=df[df['business_id']==i]
        for j in range(len(temp)):
            emptylist+=(temp['text'].iloc[j])+'\n'
        final_output[i]=emptylist
    return final_output

aggregated_reviews=lolwut(documents)

In [89]:
#converting dict to dataframe for easier manipulation
reviews=pd.DataFrame()
reviews['business_id']=list(aggregated_reviews)
reviews['text']=list(aggregated_reviews.values())

In [90]:
#Merging business names and stars with the review dataset
import pandas as pd
reviews=reviews.merge(business_details,how='left',on='business_id')
reviews.head()

,business_id,text,name,stars,review_count
0,OH3baEaklANPe1farAKgRg,"Why 4 stars? It's McDonalds, that's why. Its c...",McDonald's,2.0,28
1,UdEmYOnk2iJDY9lpEPAlJQ,"I've never had a bad experience at this place,...",Floridino's Pizza & Pasta,4.0,420
2,iAD5r5vNOzc1AP6UA5yFiQ,This place has a serious problem with unwelcom...,Bella Fresco,4.0,131
3,dNDAdeE8olF4eZaslNtBZw,Sister in town from Ontario Canada and wanted ...,Manny's Mexican Restaurant,2.5,127
4,j-6MgWWlotFZAQ23LZrTPQ,"I've been eying the Trattoria since it opened,...",Queen Creek Olive Mill Trattoria,3.5,24


In [91]:
len(reviews)

24386

In [92]:
#cleaning reviews and removing stop words to be input in the model
#Trying to use LDA to model topics for reviews
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
documents=reviews['text'] #need only the text
stoplist = list(STOPWORDS)
texts = [[word for word in document.lower().split() if (word not in stoplist) & (word.isalnum())]
         for document in documents if isinstance(document,str)] #tokenizing

In [93]:
len(texts) #checking to see if same length

24386

In [94]:
#In the above code, we tokenize all of the reviews and remove stop words
#remove words that appear only once 
#frequency of the token across ALL reviews

#counting the frequency of token across ALL reviews
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
#removing tokens that occur only once across all reviews
texts = [[token for token in text if frequency[token] > 10] for text in texts] #if frequence(token)>1

In [95]:
dictionary = corpora.Dictionary(texts) #word id-word pairs
corpus = [dictionary.doc2bow(text) for text in texts] #word id-frequence of word pairs

In [96]:
#Checking frequency of the 'contextual' words we would like to identify

import collections as c
emptylist=[]
[emptylist.extend(x) for x in texts]
len(emptylist)
count=c.Counter(emptylist)
count['candle']

93

In [71]:
#Set parameters
num_topics = 100 #can experiment with this, for the number of reviews we have, more topics==contextual words!
passes = 20
lda = LdaModel(corpus,
              id2word=dictionary,
              num_topics=num_topics,
              passes=passes)

lda.print_topics()

/Users/sanjanarosario/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


[(19,
  '0.028*"sure" + 0.027*"dinner" + 0.025*"pretty" + 0.025*"table" + 0.024*"server" + 0.021*"food" + 0.020*"seated" + 0.017*"ended" + 0.015*"experience" + 0.015*"especially"'),
 (47,
  '0.348*"hot" + 0.087*"chili" + 0.055*"dog" + 0.036*"decent" + 0.028*"visits" + 0.023*"break" + 0.020*"larger" + 0.020*"sold" + 0.017*"written" + 0.016*"beautifully"'),
 (89,
  '0.115*"fish" + 0.090*"clean" + 0.071*"cheap" + 0.053*"place" + 0.037*"lunch" + 0.032*"expect" + 0.031*"nice" + 0.030*"size" + 0.030*"coconut" + 0.026*"great"'),
 (82,
  '0.075*"staff" + 0.063*"great" + 0.050*"friendly" + 0.048*"food" + 0.038*"restaurant" + 0.030*"place" + 0.025*"helpful" + 0.024*"visit" + 0.022*"visiting" + 0.022*"owners"'),
 (64,
  '0.183*"open" + 0.073*"expensive" + 0.062*"days" + 0.052*"weeks" + 0.050*"ago" + 0.047*"comfortable" + 0.046*"hungry" + 0.038*"job" + 0.037*"casual" + 0.035*"cozy"'),
 (79,
  '0.071*"minutes" + 0.034*"time" + 0.033*"order" + 0.031*"food" + 0.030*"waited" + 0.027*"service" + 0.025*

In [ ]:
#after multiple attempts, trying LSI model, instead of LDA

from gensim.similarities.docsim import Similarity
from gensim import corpora, models, similarities
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=100)

doc=input('What are you searching for today?')
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
index = similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims[0][0]

indices=[sims[i][0] for i in range(10)]
print(reviews[['name','stars','text']].iloc[indices])

What are you searching for today?thanksgiving dinner
